In [28]:
import json
import pandas as pd

In [33]:
with open('../data/osha-limits-raw.json') as jsonfile:
    data = json.load(jsonfile)
    
data = [val for val in data if val["substance"] is not None]

def isSubLine(s):
    return s.startswith('Total d') or s.startswith("Respirable")
    
def getRealSubstance(i):
    prev = data[i-1]["substance"]
    if not isSubLine(prev):
        return prev
    else:
        return getRealSubstance(i-1)
    
def trimCeilingFlag(s):
    if s is None:
        return None
    elif(s.startswith('(C) ')):
        return s[4:]
    else:
        return s
    
for i in range(0, len(data)):
    cur = data[i]
    substance = cur["substance"]

    if(isSubLine(substance)):
        realSubstance = getRealSubstance(i)
        cur["substance"] = realSubstance + ", " + substance
        
    cur['pel_mg_m3'] = trimCeilingFlag(cur['pel_mg_m3'])
    cur['pel_ppm'] = trimCeilingFlag(cur['pel_ppm'])

        
data = [val for val in data 
        if val["pel_mg_m3"] is not None or val["pel_ppm"] is not None]

data = pd.DataFrame(data)[['substance', 'cas_no', 'pel_mg_m3', 'pel_ppm']]

data.to_csv('../data/osha-limits.csv', index=False, quoting=1)